In [1]:
import os
import os.path as osp
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.font_manager as fm

In [1]:
fontname = "DejaVu Serif"
_ = fm.findfont(fm.FontProperties(family=fontname))

fig, ax = plt.subplots()

# Set the title with the desired font name and font size
title_text = r"AFR$_\text{T}$"
fontsize = 18  # Replace with your desired font size
ax.axis('off')

# Set the title
ax.set_title(title_text, fontname=fontname, fontsize=fontsize, pad=20)

# Adjust the layout to center the title
plt.tight_layout()

# Save the figure with the title
plt.savefig("title2.png", bbox_inches='tight', dpi=500)

# Display the figure (optional)
plt.show()



NameError: name 'fm' is not defined